<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 3</font></h1>

## Problem part 3

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported')

Libraries imported


In [4]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [5]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [10]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    #print(tr_cell)
    for td_cell in tr_cell.find_all('td'):
        td_data=[]
        td_data.append(td_cell.find('b').text.strip())
        td_data.append('Not assigned')
        td_data.append('Not assigned')
        td_data_idx = 1
        for td_ele in td_cell.find_all('a'):
            td_data[td_data_idx] = td_ele.text.strip()
            td_data_idx += 1
            if td_data_idx > 2:
                break
        #print(td_data)
        df.loc[len(df)] = td_data
        #df
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park


In [12]:
df=df[df['Borough']!='Not assigned']

In [15]:
for index, row in df.iterrows():
    if row[2]=='Not assigned':
        row[2] = row[1]
    
#df[df[]=='Not assigned']=df['Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [16]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [17]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [18]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [19]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [20]:
df_merge.drop_duplicates(inplace=True)

In [21]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [22]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Queen's Park,Queen's Park


In [23]:
df_merge.shape

(101, 3)

In [24]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [29]:
#geo_df=pd.read_csv('http://cocl.us/Geospatial_data') # redirected ->>
geo_df=pd.read_csv('https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv')

In [30]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [32]:
geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,Malvern
1,M1C,43.784535,-79.160497,Scarborough,Rouge Hill
2,M1E,43.763573,-79.188711,Scarborough,Guildwood
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [33]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
2,M1E,Scarborough,Guildwood,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [34]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
45,M4R,North Toronto,North Toronto,43.715383,-79.405678
49,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
50,M4X,Downtown Toronto,St. James Town,43.667967,-79.367675
51,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
52,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636


In [35]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180604'

In [36]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

North Toronto
Rosedale
St. James Town
Church and Wellesley
Regent Park
Garden District
St. James Town
Downtown Toronto
Bay Street
Richmond
Harbourfront
Toronto Dominion Centre
Commerce Court
University of Toronto
Kensington Market
CN Tower
Downtown Toronto
First Canadian Place
Downtown Toronto


In [38]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Toronto,43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
1,North Toronto,43.715383,-79.405678,Uncle Betty's Diner,43.714452,-79.400091,Diner
2,North Toronto,43.715383,-79.405678,Civello Salon,43.715111,-79.400304,Salon / Barbershop
3,North Toronto,43.715383,-79.405678,Sushi Shop,43.713861,-79.400093,Restaurant
4,North Toronto,43.715383,-79.405678,Nailsense,43.717467,-79.400653,Spa


In [39]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,62,62,62,62,62,62
CN Tower,15,15,15,15,15,15
Church and Wellesley,78,78,78,78,78,78
Commerce Court,100,100,100,100,100,100
Downtown Toronto,175,175,175,175,175,175
First Canadian Place,100,100,100,100,100,100
Garden District,100,100,100,100,100,100
Harbourfront,100,100,100,100,100,100
Kensington Market,64,64,64,64,64,64


In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1217, 209)

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.032258,0.000000,0.000000,0.0,0.0,0.016129,0.0,0.0,0.016129,0.016129
1,CN Tower,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,Church and Wellesley,0.012821,0.012821,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012821,...,0.012821,0.012821,0.012821,0.0,0.0,0.000000,0.0,0.0,0.000000,0.025641
3,Commerce Court,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,...,0.020000,0.000000,0.000000,0.0,0.0,0.020000,0.0,0.0,0.010000,0.000000
4,Downtown Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.011429,0.000000,0.000000,0.0,0.0,0.011429,0.0,0.0,0.000000,0.005714


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bank,Japanese Restaurant,Thai Restaurant,Burger Joint,Bubble Tea Shop
1,CN Tower,Airport Service,Airport Lounge,Harbor / Marina,Rental Car Location,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Terminal,Coffee Shop
2,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store
3,Commerce Court,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,American Restaurant,Seafood Restaurant,Cocktail Bar,Deli / Bodega
4,Downtown Toronto,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Bakery,Restaurant,Park,Beer Bar,Farmers Market,Japanese Restaurant


###### Making Clusters for Neighborhood

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 4, 3, 3, 3, 3, 3, 3, 2, 0])

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,M4R,North Toronto,North Toronto,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Park,Miscellaneous Shop
49,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Trail,Playground,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
50,M4X,Downtown Toronto,St. James Town,43.667967,-79.367675,3,Coffee Shop,Café,Restaurant,Italian Restaurant,Gastropub,Bakery,Park,Cocktail Bar,Japanese Restaurant,Seafood Restaurant
51,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store
52,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,3,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Theater,Yoga Studio,Mexican Restaurant,Restaurant


In [46]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bank,Japanese Restaurant,Thai Restaurant,Burger Joint,Bubble Tea Shop
1,4,CN Tower,Airport Service,Airport Lounge,Harbor / Marina,Rental Car Location,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Terminal,Coffee Shop
2,3,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store
3,3,Commerce Court,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,American Restaurant,Seafood Restaurant,Cocktail Bar,Deli / Bodega
4,3,Downtown Toronto,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Bakery,Restaurant,Park,Beer Bar,Farmers Market,Japanese Restaurant


In [57]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))
print('ok')

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.
ok


###### Creating Map

In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters